In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import warnings 
warnings.filterwarnings('ignore')

In [3]:
#export
from fastlib.nb_03 import *

In [4]:
x_train,y_train,x_valid,y_valid = get_mnist_data()

In [5]:
x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

In [6]:
nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

In [7]:
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c_in=784, c_out=10)

In [8]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

(torch.Size([50000, 784]),
 torch.Size([50000]),
 torch.Size([10000, 784]),
 torch.Size([10000]))

In [9]:
mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

In [10]:
nfs = [8,16,32,64,64]

In [11]:
cbfs

[fastlib.nb_00.Recorder,
 functools.partial(<class 'fastlib.nb_00.AvgStatsCallback'>, <function accuracy at 0x7fc3eb0ba378>),
 fastlib.nb_01.CudaCallback,
 functools.partial(<class 'fastlib.nb_02.BatchTransformXCallback'>, <function view_tfm.<locals>._inner at 0x7fc3a9d72f28>)]

## Better CNN

In [12]:
#export
def conv_layer(ni, nf, ks=3, stride=2, bn=True, **kwargs):
    layers = [nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=not bn),
              GeneralRelu(**kwargs)]
    if bn: layers.append(nn.BatchNorm2d(nf, eps=1e-5, momentum=0.1))
    return nn.Sequential(*layers)

In [13]:
#export
def init_cnn_(m, f):
    if isinstance(m, nn.Conv2d):
        f(m.weight, a=0.1)
        if getattr(m, 'bias', None) is not None: m.bias.data.zero_()
    for l in m.children(): init_cnn_(l, f)

def init_cnn(m, uniform=False):
    f = init.kaiming_uniform_ if uniform else init.kaiming_normal_
    init_cnn_(m, f)

In [14]:
#export
import math
def prev_pow_2(x): return 2**math.floor(math.log2(x))

def get_cnn_layers(data, nfs, layer, **kwargs):
    def f(ni, nf, stride=2): return layer(ni, nf, 3, stride=stride, **kwargs)
    l1 = data.c_in
    l2 = prev_pow_2(l1*3*3)
    layers =  [f(l1  , l2  , stride=1),
               f(l2  , l2*2, stride=2),
               f(l2*2, l2*4, stride=2)]
    nfs = [l2*4] + nfs
    layers += [f(nfs[i], nfs[i+1]) for i in range(len(nfs)-1)]
    layers += [nn.AdaptiveAvgPool2d(1), Lambda(flatten), 
               nn.Linear(nfs[-1], data.c_out)]
    return layers

def get_cnn_model(data, nfs, layer, **kwargs):
    return nn.Sequential(*get_cnn_layers(data, nfs, layer, **kwargs))

### we are not using this as of yet
def get_learn_run(nfs, data, lr, layer, cbs=None, opt_func=None, **kwargs):
    model = get_cnn_model(data, nfs, layer, **kwargs)
    init_cnn(model)
    return get_runner(model, data, lr=lr, cbs=cbs, opt_func=opt_func)

In [15]:
!python notebook2script.py 04_batchnorm.ipynb

Converted 04_batchnorm.ipynb to fastlib/nb_04.py
